In [1]:
"""
NOTE: Execute this only after USFDA_Diabetes.ipynb is executed and USFDA ingredients are loaded.
"""
from neo4j import GraphDatabase
import os
import pandas as pd
import time
import json
print("Import successful")

Import successful


In [2]:
URI = os.environ["NEO4J_URI"]
USER=os.environ["NEO4J_USER_NAME"]
PASSWORD=os.environ["NEO4J_PASSWD"]
AUTH = (os.environ["NEO4J_USER_NAME"], os.environ["NEO4J_PASSWD"])

In [3]:
#Neo4J connect and Query Boilerplate

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
            
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            #response = (session.run(query, parameters))
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        
        #return pd.DataFrame([r.values() for r in response], columns=response.keys())
        return response
        
    def multi_query(self, multi_line_query, parameters=None, db=None):
        for li in multi_line_query.splitlines():
                print(li)
                result=self.query(li, parameters=None, db=None)
                print(result)

In [4]:
#Make a default connection and it should return `[<Record count(n)=0>]`
conn = Neo4jConnection(uri=URI, 
                       user=USER,
                       pwd=PASSWORD)

#if db is empty, then seed with init values 
res=conn.query('MATCH (n) RETURN count(n)')
print(res)

[<Record count(n)=13781>]


In [ ]:
# NOTE: should be executed only once per database; Alternatively, you can also copy paste this query into neo4j browser
constraint = [
"CREATE CONSTRAINT gi_ing_id FOR (gi_ing:GIIngredient) REQUIRE gi_ing.itemID IS UNIQUE;",
"CREATE CONSTRAINT gi_cat_id FOR (gi_cat:GICategory) REQUIRE gi_cat.itemID IS UNIQUE;",
"CALL db.awaitIndexes();"]

for cquery in constraint:
    try:
        res = conn.query(cquery)
        print(f"Executed successfully: {cquery}")
    except Exception as e:
        print(f"Error executing query: {cquery}\n{e}")

In [ ]:
load_ingredients = """
CALL apoc.periodic.iterate("CALL apoc.load.csv('/GlycemicIndex/GI_ing_nodes.csv') yield map as row", 
"MERGE (ing:GIIngredient {itemID: row.id}) 
ON CREATE SET ing.name = row.name, ing.productCategory = row.product_category, ing.manufacturer = row.food_manufacturer, ing.country = row.country, 
ing.GI=row.GI, ing.GL=row.GL, ing.reference=row.reference, ing.reduced_name=row.reduced_name, ing.srcUrl=row.src_url, ing.source=row.src", 
{batchSize:1000, iterateList:true, parallel:true})"""

load_category = """
CALL apoc.periodic.iterate("CALL apoc.load.csv('/GlycemicIndex/GI_cat_nodes.csv') yield map as row", 
"MERGE (c:GICategory {itemID: row.id}) 
ON CREATE SET c.name = row.name, c.srcUrl=row.src_url", 
{batchSize:1000, iterateList:true, parallel:true})"""

In [24]:
result = conn.query(load_ingredients)
print("Ingredient nodes loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))

result = conn.query(load_category)
print("GI Category nodes loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))


Ingredient nodes loaded. Time taken:0 seconds. Committed Operations: 4231. Failed Operations:0
GI Category nodes loaded. Time taken:0 seconds. Committed Operations: 3. Failed Operations:0


In [ ]:
load_gi_usfda="""
CALL apoc.periodic.iterate("CALL apoc.load.csv('/GlycemicIndex/GI_USFDA_rels.csv') yield map as row", 
"MATCH (ing:Ingredient {itemID: row.id1})
MATCH (gi_ing:GIIngredient {itemID: row.id2})
MERGE (ing)-[rel:hasGIMatch]->(gi_ing)
SET rel.GI = row.GI, 
    rel.cosine_sim = row.cos_sim", 
{batchSize:10000, iterateList:true, parallel:true})
"""

load_gi_cat="""
CALL apoc.periodic.iterate("CALL apoc.load.csv('/GlycemicIndex/GI_rels.csv') yield map as row", 
"MATCH (gi_ing:GIIngredient {itemID: row.id1})
MATCH (c:GICategory {itemID: row.id2})
MERGE (gi_ing)-[rel:has]->(c)
SET rel.GI = row.GI, 
    rel.GL = row.GL", 
{batchSize:10000, iterateList:true, parallel:true})
"""

In [26]:
result = conn.query(load_gi_usfda)
print("Relationship GIIngredient-USFDA Ingredient loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))

result = conn.query(load_gi_cat)
print("Relationship GIIngredient-GICategory loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))


Relationship GIIngredient-USFDA Ingredient loaded. Time taken:0 seconds. Committed Operations: 64462. Failed Operations:0
Relationship GIIngredient-GICategory loaded. Time taken:0 seconds. Committed Operations: 4231. Failed Operations:0
